## Analyze A/B Test Results

You may either submit your notebook through the workspace here, or you may work from your local machine and submit through the next page.  Either way assure that your code passes the project [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).  **Please save regularly.**

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

**As you work through this notebook, follow along in the classroom and answer the corresponding quiz questions associated with each question.** The labels for each classroom concept are provided for each question.  This will assure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the criteria.  As a final check, assure you meet all the criteria on the [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).

<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [ ]:
df =pd.read_csv("../input/ab_data.csv")

In [ ]:
df.head()

b. Use the cell below to find the number of rows in the dataset.

In [ ]:
df.shape

c. The number of unique users in the dataset.

In [ ]:
df['user_id'].nunique()

d. The proportion of users converted.

In [ ]:
df.converted.mean()

e. The number of times the `new_page` and `treatment` don't match.

In [ ]:
df[((df['group'] == 'treatment') == (df['landing_page'] == 'new_page')) == False].count()

f. Do any of the rows have missing values?

In [ ]:
df.info()

No its doesn't have any null values.

`2.` For the rows where **treatment** does not match with **new_page** or **control** does not match with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to figure out how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [ ]:
# Where treatment does not match with new_page or control does not match with old_page
i = df[((df['group']=='treatment') ==(df['landing_page']=='new_page')) == False].index

In [ ]:
# Drop those rows
df2 = df.drop(i)

In [ ]:
# Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [ ]:
# Number of unique users in df2
df2['user_id'].nunique()

b. There is one **user_id** repeated in **df2**.  What is it?

In [ ]:
# Number of duplicate rows
df2[df2.duplicated(['user_id'], keep=False)]

c. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [ ]:
# Drop the row and keep the first row
df2.drop_duplicates(subset ='user_id',keep ='first',inplace = True)

`4.` Use **df2** in the cells below to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [ ]:
# Prob of conversion regardless of the page
(df2.query('converted == 1').converted.count())/df2.shape[0]

b. Given that an individual was in the `control` group, what is the probability they converted?

In [ ]:
# Prob of conversion in control group
control_df = df2.query('group =="control"')
Pold = control_df['converted'].mean()
Pold

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [ ]:
# Prob of conversion in treatment group
treatment_df = df2.query('group =="treatment"')
Pnew = treatment_df['converted'].mean()

In [ ]:
Pnew 

d. What is the probability that an individual received the new page?

In [ ]:
# Prob of receiving new page
df2.query('landing_page == "new_page"').landing_page.count()/df2.shape[0]

e. Consider your results from parts (a) through (d) above, and explain below whether you think there is sufficient evidence to conclude that the new treatment page leads to more conversions.

**Your answer goes here.**

From the above results we cannot conclude that new treatment leads to more conversion. As per our analysis we can see that Pnew is smaller than P old. 

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

**Put your answer here.**

H0 = Pnew <= Pold
H1 = Pnew > Pold 

2. Assume under the null hypothesis,  𝑝𝑛𝑒𝑤  and  𝑝𝑜𝑙𝑑  both have "true" success rates equal to the converted success rate regardless of page - that is  𝑝𝑛𝑒𝑤 and  𝑝𝑜𝑙𝑑  are equal. Furthermore, assume they are equal to the converted rate in ab_data.csv regardless of the page. 


Use a sample size for each page equal to the ones in ab_data.csv. 


Perform the sampling distribution for the difference in converted between the two pages over 10,000 iterations of calculating an estimate from the null. 


Use the cells below to provide the necessary parts of this simulation. If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem. You can use Quiz 5 in the classroom to make sure you are on the right track.

a. What is the **conversion rate** for $p_{new}$ under the null? 

In [ ]:
P_new = treatment_df['converted'].mean()
P_new

b. What is the **conversion rate** for $p_{old}$ under the null? <br><br>

In [ ]:
P_old = control_df['converted'].mean()
P_old

c. What is $n_{new}$, the number of individuals in the treatment group?

In [ ]:
n_new = treatment_df.shape[0]
n_new

d. What is $n_{old}$, the number of individuals in the control group?

In [ ]:
n_old = control_df.shape[0]
n_old

e. Simulate $n_{new}$ transactions with a conversion rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [ ]:
new_page_converted = np.random.binomial(1,P_new,n_new)
new_page_converted.mean()


f. Simulate $n_{old}$ transactions with a conversion rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [ ]:
old_page_converted = np.random.binomial(1,P_old,n_old)
old_page_converted.mean()

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [ ]:
 new_page_converted.mean() - old_page_converted.mean()


h. Create 10,000 $p_{new}$ - $p_{old}$ values using the same simulation process you used in parts (a) through (g) above. Store all 10,000 values in a NumPy array called **p_diffs**.

In [ ]:
p_diffs =[]
size = df2.shape[0]
for _ in range(10000):
    new_page_converted = np.random.binomial(1,P_new, n_new)
    old_page_converted = np.random.binomial(1,P_old, n_old)
    p_diffs.append(new_page_converted.mean() - old_page_converted.mean())

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
p_diffs = np.array(p_diffs)
plt.hist(p_diffs);

In [ ]:
act_new_conversion = df2.query('converted == 1 and landing_page == "new_page"')['user_id'].nunique()/n_new

In [ ]:
act_old_conversion  = df2.query('converted == 1 and landing_page == "old_page"')['user_id'].nunique()/n_old

In [ ]:
obs_diff = act_new_conversion - act_old_conversion
obs_diff

j. What proportion of the p_diffs are greater than the actual difference observed in ab_data.csv?

In [ ]:
null_vals = np.random.normal(0, p_diffs.std(), p_diffs.size)

In [ ]:
plt.hist(null_vals)
plt.axvline(x=obs_diff,color ='red')

In [ ]:
(null_vals > obs_diff).mean()

k. Please explain using the vocabulary you've learned in this course what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

**Put your answer here.**

The value we computed in J is called P-value, which says the probability of finding the observed when null hypothesis is true.
Here P value is greater than 0.05 this means we fail to reject null hypothesis. That means old pages are better or equally performing well in conversion rates than new ones.

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
import statsmodels.api as sm

convert_old = df2.query("landing_page == 'old_page' and converted == 1").shape[0]
convert_new = df2.query("landing_page == 'new_page' and converted == 1").shape[0]
n_old = df2.query("landing_page == 'old_page'").shape[0]
n_new = df2.query("landing_page == 'new_page'").shape[0]

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](http://knowledgetack.com/python/statsmodels/proportions_ztest/) is a helpful link on using the built in.

In [ ]:
z_score, p_value = sm.stats.proportions_ztest([convert_old, convert_new], [n_old, n_new], alternative='smaller')
z_score, p_value

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

**Put your answer here.**

P value somewhat matches to the above value and its larger than 0.05 so we failed to reject null hypothesis 

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you achieved in the A/B test in Part II above can also be achieved by performing regression.<br><br> 

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

**Put your answer here.**

For all the categorical variable we must use Logistic regression by adding appropriate dummy variables.

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives. However, you first need to create in df2 a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [ ]:
df2['ab_page'] = pd.get_dummies(df['group']) ['treatment']
df2.head()

c. Use **statsmodels** to instantiate your regression model on the two columns you created in part b., then fit the model using the two columns you created in part **b.** to predict whether or not an individual converts. 

In [ ]:
df2['intercept'] = 1
logit_mod = sm.Logit(df2['converted'],df2[['intercept', 'ab_page']])
results = logit_mod.fit()
results.summary()


d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
np.exp(results.params)

In [ ]:
1/_

**Put your answer here.**
The p-value associated with logistic regression model is .190 and it is greater than .05 hence once again we failed to reject null hypotheses and unable to establish that landing page is statiscally significant conversion.And the null hypotheses is now there is no difference between the treatment and control group.Alternative hypotheses is that there is difference between between the treatment and control group.
New hypotheses are:
H0 : Pnew = Pold
H1 : Pnew != Pold